# Basic Fourier imaging

In this first tutorial / exercise session, we will explore elementary synthesis imaging concepts. We start out with a quick reminder of some Fourier theory, followed by an attempt to use VLA data to make an image of our Milky Way's centre, containing the radio source Sagittarius A* (Sgr A*), the central radio source.

## Jupyter notebooks

For the practical exercises, we will use jupyter notebooks and Python 3.7. If you have never used this before, take some time to explore the menu options. There are text cells like this one, and code cells. To execute a cell, press either `<ctrl-enter>`, `<shift-enter>`, or `<alt-enter>`. Try them all and see what happens.

To edit a cell, click on it or press `<enter>`. To leave cell-editing mode and enter notebook-editing mode, press `<esc>`. To remove a cell, go to notebook-editing mode, select the cell, and press `x`. 

## Harmonic signals / Fourier relations

We begin by deriving/recalling a bunch of relations that will come in handy during the rest of the course. As is customary, we will mostly work in the complex domain, describing for example the electric field as

$$
V(t) = \cos(2\pi\nu t) = \Re\left\lbrace \mathrm{e}^{\pm2\pi\mathrm{i}\nu t}\right\rbrace,
$$
where $\nu$ is the frequency and $t$ is time. We call
$$
V(t) =  \mathrm{e}^{\pm2\pi\mathrm{i}\nu t}
$$
the "complex voltage" to distinguish it from the actual voltage. The choice of sign here is arbitrary, but in any real world applications, whatever sign one chooses by convention at one point in for example a signal chain, determines the signs/conjugations at other points.

For the time-to-frequency Fourier transform, we use the following form:
$$
f(\nu) = \mathcal{F}^-\left\lbrace V(t)\right\rbrace = \int_{-\infty}^{+\infty} V(t) \mathrm{e}^{-2\pi\mathrm{i}\nu t}\ \mathrm{d}t
$$
with inverse
$$
V(t) = \mathcal{F}^+\left\lbrace(\nu)\right\rbrace = \int_{-\infty}^{+\infty} f(\nu) \mathrm{e}^{+2\pi\mathrm{i}\nu t}\ \mathrm{d}\nu
$$


**Remember that if it seems too simple to be true, it is most likely correct**

### 1.1) Dirac $\delta$ function

Show that the Fourier transform of a contant function is the Dirac $\delta$ function(https://en.wikipedia.org/wiki/Dirac_delta_function): 

$$\mathcal{F}^-\lbrace{f(t) = 1\rbrace} = \delta(\nu),$$
as well as
$$\mathcal{F}^+\lbrace{f(\nu) = 1\rbrace} = \delta(t).$$  

### ANSWER

$$
\mathcal{F}^-\lbrace{1\rbrace} = \delta(\nu)
$$
implies
$$
\mathcal{F}^+\lbrace{\delta(\nu)\rbrace} = 1
$$

$$\int_{-\infty}^{+\infty} \delta(\nu) \mathrm{e}^{+2\pi\mathrm{i}\nu t}\ \mathrm{d}\nu = \mathrm{e}^{+2\pi\mathrm{i}0 t},$$
which is of course equal to 1. Same for the inverse transform.

### 1.2) Shift theorem

Show through variable substitution that 
$$\mathcal{F}^-\lbrace f(t-\tau) \rbrace(\nu) 
   =\mathrm{e}^{-2\pi\mathrm{i}\nu\tau}\mathcal{F}^-\lbrace f(t) \rbrace$$, or in other words: *A shift in one domain corresponds to multiplication by a phase gradient in the other*

### ANSWER

We have
$$ \int_{-\infty}^{+\infty} f(t-\tau) \mathrm{e}^{-2\pi\mathrm{i}\nu t}\ \mathrm{d}t.$$
Let $\xi = (t -\tau)$ and therefore $\frac{\mathrm{d}t}{\mathrm{d}\xi} = 1$,
thus
$$ \int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\nu (\xi+\tau)}\ \mathrm{d}\xi$$
$$ = \int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\nu \xi}\mathrm{e}^{-2\pi\mathrm{i}\nu \tau}\ \mathrm{d}\xi$$
$$ =\mathrm{e}^{-2\pi\mathrm{i}\nu \tau}\int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\nu \xi}\ \mathrm{d}\xi$$

### 1.3) Scaling theorem

Show through variable substitution that 
$$\mathcal{F}^-\lbrace f(a\cdot t) \rbrace(\nu) 
   =|a|^{-1}\mathcal{F}^-\lbrace f(t) \rbrace (\nu/a)$$, or qualitatively: *Short things in one domain become proportionally longer in the other and v.v., while maintaining their integral.*

### ANSWER

$$ \int_{-\infty}^{+\infty} f(a\cdot t) \mathrm{e}^{-2\pi\mathrm{i}\nu t}\ \mathrm{d}t.$$

Let $\xi = a t$, thus $\frac{\mathrm{d}t}{\mathrm{d}\xi} = a^{-1}$. Therefore

$$ \int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\nu \xi/a} |a|^{-1}\ \mathrm{d}\xi$$
The absolute value appears because if $a < 0$, $\mathrm{d}\xi$ runs opposite to $\mathrm{d}t$, adding a minus sign in front of the integral.
$$= \int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\xi \nu/a} |a|^{-1}\ \mathrm{d}\xi$$
$$= |a|^{-1}\int_{-\infty}^{+\infty} f(\xi) \mathrm{e}^{-2\pi\mathrm{i}\xi (\nu/a)} \ \mathrm{d}\xi$$
Thus,
$$= |a|^{-1}\mathcal{F}^-\lbrace f(t) \rbrace (\nu/a).$$


### 1.4) Sinc function

One often has to average a range of complex numbers, for example approximately linearly varying phase errors, or a bright source's visibilities over a certain bandwidth. The ``sinc(x)`` function will invariably show up in one scaling or another. Given a certain delay $\tau$ between two radio signals, the phase difference between them as a function of frequency $\nu$, is given by $2\pi\nu\tau$.

Show that, when averaging the complex phasor $\mathrm{e}^{2\pi\mathrm{i}\nu\tau}$ between frequencies $\nu_0 - \Delta\nu/2$ and $\nu_0 + \Delta\nu/2$, the result is given by 
$$
\Delta\nu^{-1}\int_{\nu_0-\Delta\nu/2}^{\nu_0+\Delta\nu/2} \mathrm{e}^{2\pi\mathrm{i}\nu\tau}\ \mathrm{d}\nu =
\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}\frac{\sin(\pi\Delta\nu\tau)}{\pi\Delta\nu\tau}
$$

The fraction $\frac{\sin(\pi x)}{\pi x}$ is often written as $\mathrm{sinc}(x)$. `numpy` uses this definition. Note that in some conventions, one instead uses $\mathrm{sinc}(x) = \frac{\sin(x)}{x}$, therefore one should *always* pay attention what is used, and preferable forego the notation $\mathrm{sinc}(x)$ altogether.

The $\mathrm{sinc}$ function is indeed the Fourier transform of a top-hat function with unit width and unit integral. If you haven't already, check that you can derive the same result using the shift- and scaling theorems. 

Note that when performing this average, the resulting phase is the mean phase, and the amplitude is reduced. This effect is often called *decorrelation*. 



### ANSWER

$$\Delta\nu^{-1}\int_{\nu_0-\Delta\nu/2}^{\nu_0+\Delta\nu/2} \mathrm{e}^{2\pi\mathrm{i}\nu\tau}\ \mathrm{d}\nu.$$
Per the shift theorem,
$$\frac{\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}}{\Delta\nu}\int_{-\Delta\nu/2}^{+\Delta\nu/2} \mathrm{e}^{2\pi\mathrm{i}\nu\tau}\ \mathrm{d}\nu.$$

Splitting the real and imaginary parts, we get
$$\frac{\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}}{\Delta\nu}\int_{-\Delta\nu/2}^{+\Delta\nu/2} \cos{2\pi\nu\tau} + \mathrm{i}\sin{2\pi\nu\tau}\ \mathrm{d}\nu.$$
The $\sin$ term is anti-symmetric around 0, hence its integral vanishes in this symmetric interval around 0. This leaves us with
$$\frac{\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}}{\Delta\nu}\int_{-\Delta\nu/2}^{+\Delta\nu/2} \cos{2\pi\nu\tau} \ \mathrm{d}\nu,$$
which is easily integrated:
$$\frac{\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}}{\Delta\nu}\left[\frac{\sin{2\pi\nu\tau}}{2\pi\tau}\right]_{-\Delta\nu/2}^{+\Delta\nu/2}$$
$$=\frac{\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}}{\Delta\nu}\left[\frac{\sin{2\pi\Delta\nu\tau/2}}{2\pi\tau} + \frac{\sin{2\pi\Delta\nu\tau/2}}{2\pi\tau}\right]$$
$$=\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}\frac{\sin{\pi\Delta\nu\tau}}{\pi\Delta\nu\tau}$$



### ALTERNATIVE

Let $R(\nu)$ be  the top-hat function with unit width, centred at 0, and with unit integral. The problem to solve is then: 
$$\mathcal{F}^+\left\lbrace {\Delta\nu}^{-1}R\left(\frac{\nu-\nu_0}{\Delta\nu}\right)\right\rbrace(\tau)$$
$${\Delta\nu}^{-1}\mathcal{F}^+\left\lbrace R\left(\frac{\nu-\nu_0}{\Delta\nu}\right)\right\rbrace(\tau)$$
Applying the scaling theorem yields:
$$\mathcal{F}^+\left\lbrace R\left(\nu-\nu_0\right)\right\rbrace(\tau\Delta\nu)$$
And the shift theorem:
$$\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}\mathcal{F}^+\left\lbrace R\left(\nu\right)\right\rbrace(\tau\Delta\nu)$$
With
$$\mathcal{F}^+\left\lbrace R\left(\nu\right)\right\rbrace = \mathrm{sinc}(\tau)$$
we end up with
$$\mathrm{e}^{2\pi\mathrm{i}\nu_0\tau}\mathrm{sinc}(\tau\Delta\nu)$$


# Interactive part

The cell below ensures that we have all the pylab and matplotlib functions available in the main Python namespace. It also automatically reloads other python modules at every cell evaluation, which makes editing complex code elsewhere, and including it here much easier.


In [17]:
%pylab ipympl
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In the cell below we plot this sinc function for a certain bandwidth and delay. Read the code and make sure you understand each step. Play around with the sliders to build some intuition. When does the first null occur? Does that agree with what you expected?

In [20]:
import ipywidgets # Module that allows one to interact with plots

def plot_sinc(ax, bandwidth_plot_range_MHz, delay_us):
    '''
    Plot the sinc function sin(pi delta_nu tau)/(pi delta_nu tau).
    
    Parameters
    ----------
    
    ax : matplotlib.AxesSubplot
        Subplot to draw on.
        
    bandwidth_plot_range_MHz : float
        Maximum bandwidth delta_nu in the plot in MHz.
        
    delay_us : float
        Delay tau in microseconds.
    '''
    dnu_MHz = linspace(0.0, bandwidth_plot_range_MHz, 300)
    result = sinc(dnu_MHz*delay_us)

    ax.cla()
    ax.set_title(r'$\frac{\sin(\pi\Delta\nu\tau)}{\pi\Delta\nu\tau}$ for $\tau$ = %.3f $\mu$s' % delay_us)
    ax.plot(dnu_MHz, result, lw=2)
    ax.set_xlabel(r'Bandwidth $\Delta\nu$ [MHz]')
    ax.set_xlim(dnu_MHz.min(), dnu_MHz.max())
    ax.set_ylim(-0.25,1.03)
    ax.grid()

    
# Create the figure and interact with the "plot_sinc" function
fig = figure(figsize=(6,4), dpi=150)  # Create a figure using matplotlib
ax = fig.subplots(1,1)                # Create sub plots (1 row, 1 column)

layout = ipywidgets.Layout(width='80%')
desc_style =  {'description_width': 'initial'}

_ = ipywidgets.interact(
    plot_sinc,
    ax=ipywidgets.fixed(ax),
    bandwidth_plot_range_MHz=ipywidgets.FloatSlider(
        min=0.05, max=10, step=0.05, value=1,
        description=r'Plot $\Delta \nu$ [MHz]: ',
        layout=layout,
        readout_format='.3f',
        style=desc_style),
    delay_us=ipywidgets.FloatSlider(
        min=0.000, max=10, step=0.001, value=1,
        description=r'$\tau$ [$\mu$s]: ',
        layout=layout,
        readout_format='.3f',
        style=desc_style))

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=1.0, description='Plot $\\Delta \\nu$ [MHz]: ', layout=Layout(width='8…

### Heterodyne

In [14]:
time_s = linspace(0, 100e-9, 1000)
frequency_1_hz = 150e6
frequency_2_hz = 200e6
signal_1 = cos(2*pi*frequency_1_hz*time_s)
signal_2 = cos(2*pi*frequency_2_hz*time_s)

fig_1_1 = figure(dpi=150)
ax_1_1_top, ax_1_1_middle, ax_1_1_bottom = fig_1_1.subplots(3,1)
ax_1_1_top.set_title('Input signals')
ax_1_1_top.plot(time_s, signal_1)
ax_1_1_top.plot(time_s, signal_2)
ax_1_1_top.set_xticklabels([])

ax_1_1_middle.set_title('Sum')
ax_1_1_middle.plot(time_s, signal_1+signal_2)
ax_1_1_middle.set_xticklabels([])

ax_1_1_bottom.set_title('Product')
ax_1_1_bottom.plot(time_s, signal_1*signal_2)

FigureCanvasNbAgg()

cos / sin addition


In [2]:
import fourierexploration as fe

fe_gui = fe.FourierExplorerGui(figsize=(8,8), dpi=150)

shirt = fe.imread('shirt-bw.png')
pikachu = fe.imread('pikachu.png')
poliwhirl = fe.imread('poliwhirl.png')

fe_gui.interact({'Point source': fe.point_source_image,
                 'Shirt': shirt,
                 'Pikachu': pikachu,
                 'Poliwhirl': poliwhirl})

FigureCanvasNbAgg()

interactive(children=(Dropdown(description='img_scale', options={'Linear': <function linear at 0x7f4dbe147d90>…

In [3]:
s_fft = fe.fftwpack.fftshift(fe.fftwpack.ifft2(shirt(), threads=4))

Source images: 

  - single point source
  - two point sources
  - three differen circular gausians
  - a field of random elliptical gausians
  - shirt


Example Telescopes:

  - circular min / max bl
  - WSRT
  - VLA
  - GMRT
  - LOFAR Core
  - LOFAR NL
  - LOFAR Intl
  
Implement ha_range, ha_0 (only NCP projection? Set Dec?)

Parameters: Image, Telescope, DEC, HA_0, HA_RANGE

Panels: orig      orig-uv
      : Dirty     uv masked with uv-cov
      
      
Simulate noise separately?
